# Analysis of threat reports
This notebook contains cells to analyse threat reports on the threats and actors they describe.

### Build initial threat- and actor list from spreadsheet

In [2]:
import pandas as pd

file = "Threat reports.xlsx"
reports = pd.ExcelFile(file).parse('Threat reports')

In [18]:
# Build the lists
threatlist = []
actorlist = []

for index, row in reports.iterrows():
    threats = row['threats']
    if not pd.isna(threats):
        for threat in threats.split(", "):
            if threat not in threatlist:
                threatlist.append(threat)

    actors = row['actors']
    if not pd.isna(actors):
        for actor in actors.split(", "):
            if actor not in actorlist:
                actorlist.append(actor)

print("Threats:\n",threatlist)
print("Threat actors:\n",actorlist)

Threats:
 ['ransomware', 'crypto malware', 'scams', 'adware', 'fleeceware', 'phishing', '', 'coinminers', 'information stealers', 'remote access trojans', 'technical support cams', 'rootkits', 'banking trojan', 'zoombombing', 'cryptojacking', 'malspam', 'exploit kits', 'remote access', 'spyware', 'remote access trojan', 'botnet', 'remote code execution', 'supply chain attacks', 'cryptominig', 'trojans', 'exploit kit', 'information stealer', 'business email compromise', 'brand abuse', 'supply chain attack', 'data exposure on code repositories', 'targeted threats on remote work', 'DDoS', 'proxy shell', 'account compromise', 'system compromise', 'botnets', 'wipers', 'server access', 'credential harvesting', 'misconfiguration', 'malicious insider', 'vulnerability exploitation', 'brute force', 'password spraying', 'DNS attack', 'credential stuffing', 'BGP rerouting', 'network pivot', 'network infrastructure compromise', 'local code execution', 'use of stolen credentials', 'pretexting', 'mis

### PDF Scanner

In [19]:
import PyPDF2

result_list = []
reader = PyPDF2.PdfFileReader(file)
for page_number in range(0, reader.numPages):
     page = reader.getPage(page_number)
     page_content = page.extractText()
     if search_term in page_content:
          result = {
               "page": page_number,
               "content": page_content
          }
          result_list.append(result)

ModuleNotFoundError: No module named 'PyPDF2'